In [1]:
import pandas as pd
import matplotlib.pyplot as plt 
import numpy as np
import seaborn as sns
import csv
from sklearn.metrics import r2_score
from datetime import datetime
from sklearn import preprocessing
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split 
from sklearn.ensemble import GradientBoostingRegressor 
from sklearn.model_selection import RandomizedSearchCV  
from numpy.random import RandomState
from sklearn.metrics import r2_score
from lightgbm import LGBMRegressor 

In [2]:
df_for_lightgbm = pd.read_parquet('lgbmdf.parquet')

In [3]:
#title, '(no genres listed)', 'userId' featureları çıkartıldı?!?!
df_for_lightgbm = df_for_lightgbm[df_for_lightgbm.columns.difference(['title', '(no genres listed)', 'userId'])].copy()
column_names = df_for_lightgbm.columns

In [4]:
scaler = preprocessing.MinMaxScaler(feature_range=(0, 1))
df_for_lightgbm_normalized = scaler.fit_transform(df_for_lightgbm)
df_for_lightgbm_normalized = pd.DataFrame(df_for_lightgbm_normalized,columns = column_names)
df_for_lightgbm_normalized.head(5)

,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,...,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,movieId,movie_year,rating
0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.840278,0.777778
1,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.840278,1.000000
2,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.840278,0.888889
3,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.840278,0.777778
4,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.840278,0.777778


In [5]:
X = df_for_lightgbm_normalized[df_for_lightgbm_normalized.columns.difference(['rating'])]
y = df_for_lightgbm_normalized['rating']

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=31)
X_train, X_validation, y_train, y_validation = train_test_split(X_train, y_train, test_size=0.2, random_state=31)

In [7]:
"""randomstate = RandomState(66)

# lgbm_optimization_parameters
max_depth = randomstate.randint(2,15, size=4)
num_leaves = [2, 4, 8, 16]
n_estimators = randomstate.randint(100,1000, size=4)
learning_rate = [0.1, 0.2, 0.4]

lgbm_optimization_parameters = {    
    'n_estimators' : n_estimators,
    'max_depth': max_depth,
    'num_leaves': num_leaves,
    'learning_rate': learning_rate,}

# lgbm_searcher 
lgbm = LGBMRegressor(random_state = 66)
lgbm_searcher = RandomizedSearchCV(estimator = lgbm, 
                                  param_distributions = lgbm_optimization_parameters,
                                  n_iter = 5,
                                  random_state = 66)

#lgbm_search 
lgbm_search = lgbm_searcher.fit(X_train, y_train)

# Obtaining the best model from random search
lgbm_regressor = lgbm_search.best_estimator_ 
lgbm_search.best_params_ """

{'num_leaves': 16, 'n_estimators': 817, 'max_depth': 6, 'learning_rate': 0.1}

In [8]:
#61 dakika 10.4sn train süresi
y_pred_lgbm = lgbm_regressor.predict(X_validation)
R_square_lgbm = r2_score(y_validation, y_pred_lgbm) 
print(R_square_lgbm)

0.13926956062739582


In [7]:
import joblib
# save model
#joblib.dump(lgbm_search, 'lgb_minmax_scale.pkl')
# load model
gbm_pickle = joblib.load('lgb_minmax_scale.pkl')

In [8]:
lgbm_regressor = gbm_pickle.best_estimator_

In [11]:
y_pred_lgbm = lgbm_regressor.predict(X_validation)
R_square_lgbm = r2_score(y_validation, y_pred_lgbm) 
print(R_square_lgbm)

0.13926956062739582
